In [3]:

from datasets import load_dataset # huggingface datasets
dataset = load_dataset("openwebtext", num_proc=60 , cache_dir="./.cache")

/home/angelo/autoregressive/.venv/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Setting num_proc from 60 to 21 for the train split as it only contains 21 shards.
Generating train split: 100%|██████████| 8013769/8013769 [01:33<00:00, 85871.37 examples/s] 


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer= AutoTokenizer.from_pretrained("gpt2", cache_dir="./.cache")
model = AutoModelForCausalLM.from_pretrained("gpt2", cache_dir="./.cache")


In [7]:
model.num_parameters()

124439808

In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)